In [1]:
import h5py
import json
import pandas as pd
import numpy as np
import torch
import os
import pickle
import ast
from data.loader import VQADataset
import linecache
import random

In [2]:
dataset = VQADataset('train', True, False)

In [3]:
row = dataset.__getitem__(1)


In [4]:
row

(tensor([ 6,  7,  8,  9, 10, 11, 12]),
 tensor([0.2165, 0.4433, 1.0576,  ..., 0.4351, 0.1685, 0.2410]),
 tensor([1652]))

In [3]:
x = [1,2,3,4,5,10, 234, 12314, 6,10]
%timeit dataset.__getitem__(random.choice(x))

426 µs ± 38.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
